### **Setup**

In [1]:
%%bash
pip install timm -q

### **Library Imports**

In [2]:
import os
import re
import cv2
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from scipy import stats
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torchvision import models, transforms

### **Utilities and Helpers**

In [3]:
SEED: int = 42
SIZE: int = 384

LABELS: dict = {
    0 : "pure",
    1 : "broken",
    2 : "silkcut",
    3 : "discolored",
}


def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")

    
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)

### **Configuration**

In [4]:
class CFG(object):
    def __init__(self, 
                 seed: int = 42,
                 size: int = 224,
                 n_splits: int = 5,
                 batch_size: int = 16,
                 epochs: int = 25,
                 early_stopping: int = 5,
                 lr: float = 1e-4,
                 wd: float = 0.0,
                 max_lr: float = 1e-3,
                 pct_start: float = 0.2,
                 steps_per_epoch: int = 100,
                 div_factor: int = 1e3, 
                 final_div_factor: float = 1e3,
                 ):
        self.seed = seed
        self.size = size
        self.n_splits = n_splits
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.lr = lr
        self.wd = wd
        self.max_lr = max_lr
        self.pct_start = pct_start
        self.steps_per_epoch = steps_per_epoch
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.train_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.81450, 0.79094, 0.69495], [0.06435, 0.10338, 0.20791]),
            transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
            transforms.RandomHorizontalFlip(p=0.25),
            transforms.RandomVerticalFlip(p=0.25),
        ])
        self.valid_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.81450, 0.79094, 0.69495], [0.06435, 0.10338, 0.20791]),
        ])
                                
        self.save_path = "saves"
        if not os.path.exists(self.save_path): os.makedirs(self.save_path)
    
cfg = CFG(seed=SEED, size=SIZE)

### **Dataset Template**

In [5]:
class DS(Dataset):
    def __init__(
        self, 
        base_path: str,
        filenames: np.ndarray, 
        size: int,
        transform=None):
        
        self.base_path = base_path
        self.filenames = filenames
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        path = os.path.join(self.base_path, str(self.filenames[idx]) + ".png")
        image = get_image(path, self.size)
        return self.transform(image)

### **Model**

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
            
        self.model = timm.create_model(model_name="efficientnet_b4", pretrained=False)
        self.model.classifier = nn.Linear(in_features=self.model.classifier.in_features, out_features=4)
            
    def freeze(self):
        for params in self.parameters(): params.requires_grad = False

    def forward(self, x):
        return nn.LogSoftmax(dim=1)(self.model(x))

### **Predict**

In [7]:
def predict_batch(model=None, dataloader=None, path=None, device=None) -> np.ndarray:
    model.load_state_dict(torch.load(path, map_location=device)["model_state_dict"])
    model.to(device)    
    model.eval()
    
    y_pred = torch.zeros(1, 1).to(device)
    
    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            output = torch.argmax(torch.exp(model(X)), dim=1)
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

### **Submit**

In [8]:
torch.manual_seed(cfg.seed)
model = Model().to(cfg.device)

ss_df = pd.read_csv("../input/kaggle-pog-series-s01e03/corn/sample_submission.csv")
ts_filenames = ss_df["seed_id"]
ts_filenames = np.array([str(ts_filename).zfill(5) for ts_filename in ts_filenames])

ts_data_setup = DS(
    base_path="../input/kaggle-pog-series-s01e03/corn/test", 
    filenames=ts_filenames, 
    size=cfg.size,
    transform=cfg.valid_transform,
)
ts_data = DL(ts_data_setup, batch_size=cfg.batch_size, shuffle=False)

In [9]:
# Fold 1
y_pred_1_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f1/saves/ble_state_fold_1.pt",
    device=cfg.device
).reshape(1, -1)

y_pred_1_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f1/saves/bae_state_fold_1.pt",
    device=cfg.device
).reshape(1, -1)


# Fold 2
y_pred_2_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f2/saves/ble_state_fold_2.pt",
    device=cfg.device
).reshape(1, -1)

y_pred_2_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f2/saves/bae_state_fold_2.pt",
    device=cfg.device
).reshape(1, -1)


# Fold 3
y_pred_3_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f3/saves/ble_state_fold_3.pt",
    device=cfg.device
).reshape(1, -1)

y_pred_3_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f3/saves/bae_state_fold_3.pt",
    device=cfg.device
).reshape(1, -1)


# Fold 4
y_pred_4_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f4/saves/ble_state_fold_4.pt",
    device=cfg.device
).reshape(1, -1)

y_pred_4_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f4/saves/bae_state_fold_4.pt",
    device=cfg.device
).reshape(1, -1)


# Fold 5
y_pred_5_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f5/saves/ble_state_fold_5.pt",
    device=cfg.device
).reshape(1, -1)

y_pred_5_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="../input/kps-en4-a384-f5/saves/bae_state_fold_5.pt",
    device=cfg.device
).reshape(1, -1)

In [10]:
y_pred = np.concatenate(
    (y_pred_1_l,
     y_pred_1_a, 
     y_pred_2_l,
     y_pred_2_a,
     y_pred_3_l,
     y_pred_3_a,
     y_pred_4_l,
     y_pred_4_a,
     y_pred_5_l,
     y_pred_5_a,
    ), axis=0)
y_pred = stats.mode(y_pred)[0]
y_pred = list(y_pred.astype("uint8").reshape(-1))
y_pred = [LABELS[y_pred[i]] for i in range(len(y_pred))]

ss_df["label"] = y_pred
ss_df.to_csv("submission.csv", index=False)